In [12]:
import pandas as pd
import numpy as np
import difflib
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

import nltk
from nltk.corpus import stopwords

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [13]:
# constants
DATA_PATH = '../data/'
EDUCATION_PATH = DATA_PATH + 'education/'

In [19]:
try:
    df_edu_all = pd.read_pickle(EDUCATION_PATH + 'DOE-college-data.pickle')
except FileNotFoundError:
    df_edu_all = pd.read_csv(EDUCATION_PATH + 'Most-Recent-Cohorts-All-Data-Elements.csv',encoding = 'ISO-8859-1')
    df_edu_all.to_pickle(EDUCATION_PATH +'DOE-college-data.pickle')

In [20]:
np.shape(df_edu_all)

(7593, 1777)

In [67]:
# creating list of column names found in ../data/education/CollegeScorecardDataDictionary.xlsx
columns = ['PCIP0' + str(i) if len(str(i)) == 1 else 'PCIP' + str(i) for i in range(1,55)]
print(len(columns))
columns.append('MN_EARN_WNE_P10')
columns.append('MD_EARN_WNE_P10')
columns.append('ADM_RATE')
columns.insert(0,'INSTNM')

54


In [25]:
df_edu = df_edu_all.filter(columns, axis=1)

In [48]:
np.shape(df_edu)

(7593, 42)

In [60]:
np.sum(df_edu.iloc[:1, 1:39].values)

1.0001